# Programowanie wielowątkowe

Od GIL po bezpieczną współbieżność i `concurrent.futures`.


## Cele
- zrozumieć ograniczenia GIL i kiedy wątki mają sens
- nauczyć się korzystać z modułów `threading` i `concurrent.futures`
- zarządzać współdzielonym stanem przy pomocy blokad


## Wątki i GIL
Python umożliwia współbieżność I/O mimo blokady interpretacji.


In [2]:
import threading

print("Liczba aktywnych wątków:", threading.active_count())


Liczba aktywnych wątków: 9


## Podstawowy przykład z `threading`
Tworzymy wątki do zadań I/O, używając `Lock` dla bezpieczeństwa.


In [3]:
import time
from threading import Thread, Lock

lock = Lock()

results = []

def fetch(name: str, delay: float):
    time.sleep(delay)  # symulacja oczekiwania I/O
    with lock:  # sekcja krytyczna
        results.append((name, delay))

threads = [Thread(target=fetch, args=(f"t{i}", i * 0.1)) for i in range(3)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

print(results)


[('t0', 0.0), ('t1', 0.1), ('t2', 0.2)]


## `ThreadPoolExecutor`
Wyższy poziom abstrakcji do równoległego uruchamiania funkcji.


In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed

urls = ["https://example.com", "https://python.org", "https://openai.com"]

def download(url):
    time.sleep(0.1)
    return url, len(url)

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(download, url) for url in urls]
    for future in as_completed(futures):
        print(future.result())


('https://example.com', 19)
('https://python.org', 18)
('https://openai.com', 18)


**Podsumowanie:** Wątki przyspieszają operacje I/O, lecz wymagają ochrony stanu.

**Pytanie kontrolne:** Dlaczego wątki nie przyspieszą obliczeń CPU-bound w CPythonie?


### 🧩 Zadanie 1
Zaimplementuj pulę wątków, która pobiera dane z listy URL-i (symulacja `sleep`) i mierzy całkowity czas wykonania.


In [5]:
# Rozwiązanie Zadania 1
import time
from concurrent.futures import ThreadPoolExecutor

urls = [f"https://service/{i}" for i in range(5)]

def fake_request(url):
    time.sleep(0.2)
    return url

start = time.perf_counter()
with ThreadPoolExecutor(max_workers=5) as executor:
    list(executor.map(fake_request, urls))
elapsed = time.perf_counter() - start
print(f"Czas całkowity: {elapsed:.2f}s")


Czas całkowity: 0.21s


### 🧩 Zadanie 2
Stwórz licznik współdzielony między wątkami, zabezpieczony blokadą, i zademonstruj
problem bez blokady.


In [6]:
# Rozwiązanie Zadania 2
from threading import Thread, Lock

counter = 0
lock = Lock()

def increment(use_lock):
    global counter
    for _ in range(100_000):
        if use_lock:
            with lock:
                counter += 1
        else:
            counter += 1

for mode in (False, True):
    counter = 0
    threads = [Thread(target=increment, args=(mode,)) for _ in range(2)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    print("Z blokadą" if mode else "Bez blokady", counter)


Bez blokady 200000
Z blokadą 200000
